___
<h1><center>  Clustering</center></h1>

___

## Customer Segmentation of   Product company

###Q3a Description :

The objective of the problem is to analyze the user buying behavior based on their historical data.



In [ ]:
from IPython.core.display import display, HTML
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from scipy.stats import boxcox, probplot, norm, shapiro
from sklearn.preprocessing import PowerTransformer, MinMaxScaler
from sklearn.cluster import KMeans
import os
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Main function for plot. Usefull for other case of clustering.

def comprueba_normalidad(df, return_type='axes', title='Normalization'):
    '''
    '''
    fig_tot = (len(df.columns))
    fig_por_fila = 3.
    tamanio_fig = 4.
    num_filas = int( np.ceil(fig_tot/fig_por_fila) )    
    plt.figure( figsize=( fig_por_fila*tamanio_fig+5, num_filas*tamanio_fig+2 ) )
    c = 0 
    shapiro_test = {}
    lambdas = {}
    for i, col in enumerate(df.columns):
        ax = plt.subplot(num_filas, fig_por_fila, i+1)
        probplot(x = df[df.columns[i]], dist=norm, plot=ax)
        plt.title(df.columns[i])
        shapiro_test[df.columns[i]] = shapiro(df[df.columns[i]])
    plt.suptitle(title)
    plt.show()
    shapiro_test = pd.DataFrame(shapiro_test, index=['Test Statistic', 'p-value']).transpose()
    return shapiro_test

<h1><center> First Step </center></h1>

In [ ]:
os.listdir()

In [ ]:
XY = pd.read_csv('../input/customerbehaviour/q3a-product-user.csv')

In [ ]:
XY.head(2)

In [ ]:
XY.describe()

In [ ]:
XY.info()

<h1><center> GRAPHICS </center></h1>

I save in a XY_cuants ** with  the non-missing and numerical features 

In [ ]:
plt.figure(1 , figsize = (15 , 5))
sns.countplot(x = 'STATE' , data = XY)
plt.show()

In [ ]:
XY_cuants = XY[['AGE', 'EDU', 'YEARS_EMPLOYED', 'INCOME', 
       'CARD_DEBT', 'OTHER_DEBT',  'DEBT_INCOME_RATIO']].copy()

In [ ]:
XY_normalizado = (XY_cuants-XY_cuants.mean())/XY_cuants.std()
# This function shows more ordered graph. 
 

In [ ]:
plt.figure(figsize=(14,6))
ax = sns.boxplot(data=XY_normalizado)
ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
plt.title(u'Representation of (Normarlized) Product User Information ')
plt.ylabel('The Normalized Value ')
_ = plt.xlabel('Product User Information')

In [ ]:
## Representation of the distributions of the variables using histograms.

In [ ]:
plt.figure(figsize=(20,20))
n = 0
for i, column in enumerate(XY_cuants.columns):
    n+=1
    plt.subplot(5, 5, n)
    sns.distplot(XY_cuants[column], bins=30)
    plt.title('Distribution of Product User Info'.format(column))
plt.show()

<h1><center> Representation of the correlation Matrix.

In [ ]:
matrix_corr = XY_cuants.corr(method='pearson')
n_ticks = len(XY_cuants.columns)
plt.figure( figsize=(15, 10) )
plt.xticks(range(n_ticks), XY.columns, rotation='vertical')
plt.yticks(range(n_ticks), XY.columns)
plt.colorbar(plt.imshow(matrix_corr, interpolation='nearest', 
                            vmin=-1., vmax=1., 
                            cmap=plt.get_cmap('Blues')))

In [ ]:
plt.rcParams['figure.figsize'] = (15, 8)
sns.heatmap(XY_cuants.corr(), cmap = 'Wistia', annot = True)
plt.title('Heatmap for the Data', fontsize = 20)
plt.show()

<h1><center> Data Transformation to find the hypothesis.

# Data normalization:

Variable normalization is the process in which a variable is transformed to follow a normal or Gaussian distribution.

In general, we will only want to normalize the data if we are going to use a machine learning algorithm or a statistical technique that assumes that the data is distributed in a Gaussian or normal way. For example, student's t tests, ANOVAs, linear regressions, logistic regressions, linear discriminant analysis (LDA), k-means, etc.

Among the ways to transform a variable to normal are methods such as the Box-Cox transformation or the Yeo-Johnson method.

The following graphs represent the <a href='https://es.wikipedia.org/wiki/Gr%C3%A1fico_Q-Q'>Q-Q Plot</a>, which is a graph that compares between two distributions. In this case, each of the variables with a normal distribution. If they follow the same distribution, the points fall close to the red line.

In [ ]:
shapiro_test = comprueba_normalidad(XY_cuants, title='Variable Normalization')

In [ ]:
shapiro_test

All variables are statistically significantly not distributed as a normal.

** Shapiro-Wilk test: ** If the p-value is less than a significance level , it is concluded that the distribution does not come from a normal one.


## Pre-scaling the data:

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
X_escalado = scaler.fit_transform(XY_cuants)

In [ ]:
shapiro_test

<h1><center> Segmentation using K-means clustering:

In [ ]:
# Now, with the next code, we are looking for the best number of cluster for our dataset.

In [ ]:
cluster_range = range(1,20)
cluster_wss=[] 
for cluster in cluster_range:
    model = KMeans(cluster)
    model.fit(X_escalado)
    cluster_wss.append(model.inertia_)

In [ ]:
plt.figure(figsize=[10,6])
plt.title('WSS curve to find the optimal value of clusters ')
plt.xlabel('# groups')
plt.ylabel('WSS')
plt.plot(list(cluster_range),cluster_wss,marker='o')
plt.show()


The graph assumes the optimal point when the curve creates a bend. In this case it would be about 4-6 groups.

We will choose the number of groups at 6, but what is usually done is to try several and see if the final results make sense from a business point of view, as I will comment later.

In [ ]:
model = KMeans(n_clusters=6,random_state=0)
model.fit(X_escalado)


### I predict and get customers with your prediction

I create a dataframe with all the variables and a new one that is the prediction of the assigned cluster:

In [ ]:
#Original Dataset with the predictions
df_total = XY_cuants.copy()
df_total['cluster']=model.predict(X_escalado)
df_total[:2]

In [ ]:
df_total.cluster.value_counts().plot(kind='bar', figsize=(10,4))
plt.title('Count of users per group')
plt.xlabel('Group')
_ = plt.ylabel('Counting')

In [ ]:
#Here, we coud see our clients inside of a cluster

# Now, we have to obtain the characteristic of each group to find the hiden information inside our DF and give value to our analysis.

I also get a dataframe with the means of the variables in each group. This would represent each of the groups.

This is very necessary since the actions that the objective of this problem would be to do actions to each of the groups separately. For this, it is very important to know what each group is like, in order to act differently.

In [ ]:
descriptivos_grupos = df_total.groupby(['cluster'],as_index=False).mean()
descriptivos_grupos

# I explain the groups using the means of each variable per group: ¶

I distingusih the group by the income 

In [ ]:
df_total.groupby('cluster').mean().plot(kind='bar', figsize=(15,7))
plt.xlabel(u'Cluster Number')
_ = plt.ylabel('Customer Characteristics')
plt.title('Customer Characteristics based on Income')

Finally, for each of the groups, I obtain their user behaviors or characteristics based on different income level


In [ ]:
df_total[:2]